<a href="https://colab.research.google.com/github/PinaxX/Linkedin-Scraper/blob/main/Linkedin_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook can scrape posts from *Linkedin* using user-defined tags. Since there are no official APIs introduced yet, this is the best we can do.

Works fine in 02/28/2021. If *Linkedin* changes URLs or login strategies it'll need modifications.

There are 4 parameters that you need to modify: *username*, *password*, *hashtags* and *SCROLL_PAUSE_TIME*. You can find the first 2 in the second code cell and the other 2 can be found in the third cell.

The scraped data consists of post's text, post's relative date and its author. The last cell will save the data into a CSV file.


# Dependencies:

In [ ]:
%%capture
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=chrome_options)

# Login:

In [ ]:
username = 'Your Email'
password = 'Your Password'

wd.get("https://www.linkedin.com/")
wd.find_element_by_id("session_key").send_keys(username)
wd.find_element_by_id("session_password").send_keys(password)
wd.find_element_by_class_name("sign-in-form__submit-button").click()

# Scrape:

In [ ]:
hashtags = {'machinelearning', 'datascience'}
SCROLL_PAUSE_TIME = 3

data = []

for hashtag in hashtags:
  count = 0
  wd.get("https://www.linkedin.com/feed/hashtag/?keywords=" + hashtag)
  time.sleep(SCROLL_PAUSE_TIME)
  last_height = wd.execute_script("return document.body.scrollHeight")

  while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      break
    last_height = new_height

  texts = soup.findAll("span", {"class": "break-words"})
  names = soup.findAll("span", {"class": "feed-shared-actor__name t-14 t-bold hoverable-link-text t-black"})
  dates = soup.findAll("span", {"class": "feed-shared-actor__sub-description t-12 t-normal t-black--light"})
  for i in range(len(texts)):
    texts[i] = BeautifulSoup(str(texts[i]).replace('<br/>', ' <enter> ').replace(',', ' ').replace('\n', ' <enter> '),
                             'html.parser').text
    names[i] = BeautifulSoup(str(names[i]).replace('<br/>', '').replace(',', ' ').replace('\n', ''), 'html.parser').text
    dates[i] = BeautifulSoup(str(dates[i]).replace('<br/>', '').replace(',', ' ').replace('\n', ''), 'html.parser').text
    dates[i] = dates[i][dates[i].find('  ') + 2:]
    data.append([hashtag, dates[i], names[i], texts[i]])
    count += 1

  print(hashtag + ': ' + str(count))

wd.close()

machinelearning: 533
datascience: 473


# Save Data:

In [ ]:
df = pd.DataFrame(data, columns=['Hashtag', 'Date', 'Name', 'Text'])
df.to_csv('data.csv', index=False)
df.head()

,Hashtag,Date,Name,Text
0,machinelearning,35 minutes ago,Vincent Boucher,Investigating the Limitations of the Transfo...
1,machinelearning,8 minutes ago,Arnav Gupta,Very important thing to understand for those...
2,machinelearning,5 days ago,Sabya Dasgupta Ph.D,If LSTM's were great at predicting BTC price...
3,machinelearning,18 minutes ago,Maz'ar Ul Haq,The Future Dogmas That Will Challenge The Fi...
4,machinelearning,1 day ago,Hashem Al-Ghaili,‘Deep Nostalgia’ uses AI to turn Old Photos ...
